In [ ]:
from allensdk.api.queries.image_download_api import ImageDownloadApi
from allensdk.api.queries.ontologies_api import OntologiesApi
from allensdk.api.queries.svg_api import SvgApi
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as et 
import re
import os
import logging
import urllib.request
from allensdk.config.manifest import Manifest
from pathlib import Path

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


image_api = ImageDownloadApi()
svg_api = SvgApi()

atlas_id = 1

# image_api.section_image_query(section_data_set_id) is the analogous method for section data sets
atlas_image_records = image_api.atlas_image_query(atlas_id)

# this returns a list of dictionaries. Let's convert it to a pandas dataframe
atlas_image_dataframe = pd.DataFrame(atlas_image_records)[['id']]

# and use the .head() method to display the first few rows
atlas_image_dataframe.head()

In [ ]:
atlas_image_dataframe.shape

In [ ]:
for i in atlas_image_dataframe.id:
    stop = False
    #Our internet was to slow so we've to try multiple time
    while stop == False:        
        try:
            image_api.download_atlas_image(i, 'JPG_Coronal/'+str(i)+'.jpg', annotation=False, downsample=0)
            svg_api.download_svg(i, file_path='SVG_Coronal/'+str(i)+'.svg')
            stop = True
        except:
            None

In [ ]:
onto_ = OntologiesApi()

struc_dics_list = onto_.get_structures(structure_graph_ids=1, num_rows='all')

atlas_id = [i['id'] for i in struc_dics_list]
depth = [i['depth'] for i in struc_dics_list]
name = [i['name'] for i in struc_dics_list]
parent_structure_id = [i['parent_structure_id'] for i in struc_dics_list]
color_hex_triplet = [i['color_hex_triplet'] for i in struc_dics_list]

struc_dataFrame = (pd
                   .DataFrame({'atlas_id':atlas_id,
                               'name':name,
                               'depth':depth,
                               'parent_structure_id':parent_structure_id,
                               'color_hex_triplet':color_hex_triplet})
                   .set_index('atlas_id'))

struc_dataFrame.to_csv('complet.csv')

In [2]:
logging.getLogger('allensdk.api.api.retrieve_file_over_http').disabled = True

def get_values(url,id_):
    Gene_xml = re.sub('&.+[0-9]+;', '', requests.get(url, allow_redirects=True).content.decode("utf-8"))
    gene_root = (et.fromstring(Gene_xml)).getchildren()[0]
    return([int(i.findtext(id_))for i in gene_root])

#Function to dwnloads debends on Genes:

# Step one retreve all section data for the specific Gene:
#define ULR
rootDir = Path()
maskSvgDir = rootDir / "Genes"
maskSvgDir.mkdir(parents=True, exist_ok=True)

def download_images_allen(gene):
    url_one = "http://api.brain-map.org/api/v2/data/query.xml?criteria=model::SectionDataSet,rma::criteria,[failed$eq'false'],products[abbreviation$eq'Mouse'],plane_of_section[name$eq'coronal'],genes[acronym$eq'"+gene+"']"


    SectionDataSets  = get_values(url_one,"id")

    print(gene)

    url = 'http://api.brain-map.org/api/v2/data/query.xml?criteria=model::SectionImage,rma::criteria,[data_set_id$eq{}]'

    all_urls_sections = [url.format(i) for i in SectionDataSets]

    list_c = [get_values(i,"id") for i in all_urls_sections]

    list_flat = [item for sublist in list_c for item in sublist]

    print('Finished list!')
    
    
    for i in list_flat:
        try :
            file_name = 'Genes/'+str(i) + ".jpg"
            Manifest.safe_make_parent_dirs(file_name)
            image_api.download_section_image(i, file_path=file_name, downsample=0)
        except:
            print("Imaged "+str(i)+" Passed!")


In [3]:
genes = pd.read_csv("genes.txt",header=None,names=["Gene"]).Gene.unique()

In [4]:
for g in genes:
    download_images_allen(g)

Pak7


KeyboardInterrupt: 